# Noise minibatch

In [1]:
# PYTHON 2
from __future__ import print_function
import numpy as np
import pickle
from decagon.deep.minibatch import EdgeMinibatchIterator

In [2]:
noise = 0.05

In [3]:
mb_file = 'data/data_structures/MINIBATCH/MINIBATCH_toy_genes_500_drugs_400_se_4_batchsize_512_valsize_0.15'
with open(mb_file, 'rb') as f:
    minibatch = pickle.load(f)

In [4]:
for edge in minibatch.test_edges:
    for p in range(np.shape(minibatch.test_edges[edge])[0]):
        data_true = minibatch.test_edges[edge][p]
        data_false = minibatch.test_edges_false[edge][p]
        assert len(data_true)==len(data_false), 'Dimension of true and false mismatch'
        l = len(data_true)
        full_idx = np.arange(2*l)
        n_idx = np.floor(noise*2*l).astype(np.int)
        idx = np.random.choice(full_idx,size=n_idx)
        col = np.zeros(len(idx))
        # We choose a convention that indices from l to 2l are from the true dataset
        for i in range(len(idx)):
            if idx[i] >= l:
                idx[i] = idx[i]-l
                col[i] = 1
        col = col.astype(np.bool)
        false_to_true = idx[~col]
        true_to_false = idx[col]
        data_true = np.concatenate((data_true,data_false[false_to_true]),axis=0)
        data_false = np.concatenate((data_false,data_true[true_to_false]),axis=0)
        minibatch.test_edges[edge][p] = np.delete(data_true, true_to_false, axis=0)
        minibatch.test_edges_false[edge][p] = np.delete(data_false, false_to_true, axis=0)

In [7]:
out_file = mb_file + '_noise_' + str(noise)
print(out_file)

data/data_structures/MINIBATCH/MINIBATCH_toy_genes_500_drugs_400_se_4_batchsize_512_valsize_0.15_noise_0.05


In [8]:
with open(out_file,'wb') as f:
    pickle.dump(minibatch, f, protocol=2)